In [ ]:
import pandas as pd
import numpy as np
import datetime as dt
import snscrape.modules.twitter as sntwitter
from emoji import UNICODE_EMOJI # Read documentation
import requests
import bs4
import nltk
from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')

**Defining tickers**

In [63]:
ticker = "AAPL"

**Twitter scraping example**

In [113]:
def twitter_scrape(ticker, max_tweets):
    all_tweets = []
    all_emojis = []
    for i, tweet in enumerate(sntwitter.TwitterSearchScraper(ticker).get_items()):
        if i > max_tweets:
            break
        all_tweets.append(tweet.content)

    for i in all_tweets:
        for element in i:
            if element in UNICODE_EMOJI['en']:
                all_emojis.append(element)

    return (all_emojis, all_tweets)

all_emojis = twitter_scrape(ticker, 100)[0]
all_tweets = twitter_scrape(ticker, 100)[1]

**Obtaining S&P 500 tickers:**

In [65]:
wikipedia=pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
first_table = wikipedia[0]
tickers = first_table["Symbol"]
sp_tickers = [i for i in tickers]

**Scraping for emoji category data from the web:**

In [66]:
url = "http://kt.ijs.si/data/Emoji_sentiment_ranking/"
response = requests.get(url)
html_parsed = bs4.BeautifulSoup(response.text, 'html.parser')


**Classifying the main table into sub-tables with individual emojis (TRs):**

In [67]:
tables = html_parsed.find_all("tr")

**Scraping emoji symbols:**

In [68]:
emoji_list = []

for row in tables:
    string_row = str(row)
    emoji = string_row[(string_row.find("<tr><td>")+8):(string_row.find("<tr><td>")+9)]
    emoji_list.append(emoji)

**Scraping sentiment scores:**

In [69]:
sentiment_scores = []

for row in tables:
    string_row = str(row)
    if string_row[string_row.find("sentiment score: ")+17] == "-":
        score = string_row[(string_row.find("sentiment score: ")+17):(string_row.find("sentiment score: ")+23)]
    else:
        score = string_row[(string_row.find("sentiment score: ")+17):(string_row.find("sentiment score: ")+22)]
    sentiment_scores.append(score)

**Creating a dictionary:**

In [95]:
score_dict = {}
for i in range(1,752):
    score_dict[emoji_list[i]] = sentiment_scores[i]

**Calculating sentiment scores from actual tweets:**

In [154]:
def emoji_score(all_emojis):
    total_score = 0
    for emoji in all_emojis:
        if emoji in score_dict.keys():
            total_score = total_score + float(score_dict[emoji])
    return round(total_score,2)

emoji_score1 = emoji_score(all_emojis)
print("Current sentiment score for", ticker, "is", round(emoji_score1, 3))

Current sentiment score for ZTS is 14.81


**Starting with text analysis using NLTK:**

Source: https://realpython.com/python-nltk-sentiment-analysis/

**Importing stop-words, such as "and", "or" and "but" to filter them out:**

In [ ]:
nltk.download('stopwords')
stopwords = nltk.corpus.stopwords.words("english")

**Creating tokenized lists of tweet words:**

In [73]:
tokenized_list = []
for x in all_tweets:
    split_tweet = x.split()
    for word in split_tweet:
        if word.isalpha():
            if word.lower() not in stopwords:
                tokenized_list.append(word.lower())


**Looking at the most frequent words:**

In [ ]:
x = nltk.FreqDist(tokenized_list)
x.tabulate(10)

**Looking at the most frequent three-word collocations:**

In [ ]:
y = nltk.collocations.TrigramCollocationFinder.from_words(tokenized_list)
y.ngram_fd.most_common(5)

**Performing sentiment analysis using VADER library:**

In [76]:
sent = SentimentIntensityAnalyzer()

**Counting the number of positive, negative and neutral tweets:**

In [128]:
def text_score(all_tweets):
    positive = 0
    negative = 0
    neutral = 0
    for x in all_tweets:
        tweet_score = sent.polarity_scores(x)
        if tweet_score["compound"] >= 0.3:
            positive += 1
        elif tweet_score["compound"] > -0.3:
            neutral += 1
        else:
            negative += 1
    return [positive, neutral, negative]

**Providing a summary of the sentiment, that is, emoji score + number of positive/negative/neutral tweets:**

In [130]:
emoji_score2 = emoji_score(all_emojis)
text_score2 = text_score(all_tweets)

print(emoji_score2, text_score2)

14.809 [17, 76, 8]


**Providing a summary of these scores for all S&P 500 tickers:**

In [ ]:
sp500_scores = pd.DataFrame()

for ticker in sp_tickers:
    data = twitter_scrape(ticker, 10)
    emojis = data[0]
    tweets = data[1]
    sp500_scores.loc[ticker, "Ticker"] = ticker
    sp500_scores.loc[ticker, "Emoji Score"] = emoji_score(emojis)
    sp500_scores.loc[ticker, "Positive"] = text_score(tweets)[0]
    sp500_scores.loc[ticker, "Neutral"] = text_score(tweets)[1]
    sp500_scores.loc[ticker, "Negative"] = text_score(tweets)[2]

**Sorting by the highest Emoji Scores, as an example:**

In [155]:
sp500_scores.sort_values(by=["Emoji Score"], ascending=False)

,Ticker,Emoji Score,Positive,Neutral,Negative
TFC,TFC,20.792,2.0,9.0,0.0
LDOS,LDOS,15.600,0.0,8.0,3.0
HCA,HCA,14.135,4.0,6.0,1.0
SIVB,SIVB,11.250,1.0,10.0,0.0
T,T,11.077,4.0,6.0,1.0
...,...,...,...,...,...
ETN,ETN,-0.603,1.0,8.0,2.0
CAT,CAT,-0.773,2.0,8.0,1.0
NOW,NOW,-1.126,8.0,2.0,1.0
ANET,ANET,-1.649,1.0,9.0,1.0
